In [9]:
import timm
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
from timm.data import resolve_model_data_config, create_transform
import random
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from common.datasets import LandmarkDataset
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

##### Environment variables

In [10]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATASET_PATH = "D:/Datasets/landmark-recognition-2021"
TRAIN_DIR = f"{DATASET_PATH}/train"
TEST_DIR = f"{DATASET_PATH}/test"

##### Hyperparameters

In [11]:
SEED = 42
EPOCHS = 1
BATCH_SIZE = 16
N_WORKERS = 2

##### Training IDs table

In [12]:
train_df = pd.read_csv(f"{DATASET_PATH}/train.csv")
train_df.head()

,id,landmark_id
0,17660ef415d37059,1
1,92b6290d571448f6,1
2,cd41bf948edc0340,1
3,fb09f1e98c6d2f70,1
4,25c9dfc7ea69838d,7


##### Seeding

In [13]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.benchmark = False


seed_everything(SEED)

##### Training/Test datasets

In [14]:
USE_SUBSET = True
SUBSET_SIZE = 100

labels = dict(zip(train_df["id"], train_df["landmark_id"]))

unique_landmarks = sorted(train_df["landmark_id"].unique())
landmark2idx_global = {l: i for i, l in enumerate(unique_landmarks)}
labels_global = {img_id: landmark2idx_global[l_id] for img_id, l_id in labels.items()}

train_ids, test_ids = train_test_split(
    train_df["id"].tolist(),
    test_size=0.2,
    random_state=SEED,
    stratify=train_df["landmark_id"],
)

if USE_SUBSET:
    train_ids_small = train_ids[:SUBSET_SIZE]
    test_ids_small = test_ids[:SUBSET_SIZE]
    partition = {"train": train_ids_small, "test": test_ids_small}

    unique_landmarks_subset = sorted(
        set(labels_global[img_id] for img_id in partition["train"] + partition["test"])
    )
    landmark2idx_subset = {l: i for i, l in enumerate(unique_landmarks_subset)}
    labels_subset = {
        img_id: landmark2idx_subset[labels_global[img_id]]
        for img_id in partition["train"] + partition["test"]
    }

    labels_to_use = labels_subset
else:
    partition = {"train": train_ids, "test": test_ids}
    labels_to_use = labels_global

print(f"Training IDs: {len(partition['train'])}")
print(f"Test IDs: {len(partition['test'])}")
print(f"Total labels used: {len(labels_to_use)}")

Training IDs: 100
Test IDs: 100
Total labels used: 200


In [15]:
params = {"batch_size": BATCH_SIZE, "shuffle": True, "num_workers": N_WORKERS}

model = timm.create_model("efficientnetv2_m", pretrained=False)
model_config = timm.data.resolve_model_data_config(model)
transform = timm.data.create_transform(**model_config)
model.to(DEVICE)

train_set = LandmarkDataset(partition["train"], labels_to_use, directory=TRAIN_DIR, transform=transform)
test_set = LandmarkDataset(partition["test"], labels_to_use, directory=TEST_DIR, transform=transform)

train_loader = DataLoader(train_set, **params)
test_loader = DataLoader(test_set, **params)

len(train_set), len(test_set)

(100, 100)

##### Training loop

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{EPOCHS}]")
    for images, labels in loop:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        loop.set_postfix(loss=running_loss / total, accuracy=100.0 * correct / total)

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%")

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_loss /= val_total
    val_acc = 100.0 * val_correct / val_total
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%\n")

Epoch [1/1]: 100%|██████████| 7/7 [00:35<00:00,  5.10s/it, accuracy=0, loss=6.92]


Train Loss: 6.9224, Train Acc: 0.00%


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Admin\PycharmProjects\kaggle-projects\src\common\datasets.py", line 49, in __getitem__
    feature = Image.open(path).convert(self.colorspace)
              ^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\PycharmProjects\kaggle-projects\.venv\Lib\site-packages\PIL\Image.py", line 3513, in open
    fp = builtins.open(filename, "rb")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'D:/Datasets/landmark-recognition-2021/test\\6/5/9/659b31d2915a01b4.jpg'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Admin\PycharmProjects\kaggle-projects\.venv\Lib\site-packages\torch\utils\data\_utils\worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\PycharmPro